In [1]:
import pandas as pd
import datetime
path = f'../../../siads591 data/debris_yearly_gabbard_pkl/'

In [2]:
dfs = []
freq = '1D'
# for y in range(1999,2002):
for y in range(2007,2010):
    df = pd.read_pickle(f'{path}{y}.pkl.gz', compression="gzip")
    dfs.append(df)
raw_df = pd.concat(dfs)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8221261 entries, 2007-01-01 00:03:24.443999 to 2009-12-31 23:59:57.716448
Data columns (total 4 columns):
 #   Column        Dtype  
---  ------        -----  
 0   NORAD_CAT_ID  uint32 
 1   PERIOD        float32
 2   APOAPSIS      float32
 3   PERIAPSIS     float32
dtypes: float32(3), uint32(1)
memory usage: 188.2 MB


In [3]:
df = raw_df.reset_index().groupby(['NORAD_CAT_ID', pd.Grouper(key='EPOCH', freq=freq)]).mean()
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5764264 entries, (51, Timestamp('2007-01-01 00:00:00', freq='D')) to (36277, Timestamp('2009-12-29 00:00:00', freq='D'))
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   PERIOD     float32
 1   APOAPSIS   float32
 2   PERIAPSIS  float32
dtypes: float32(3)
memory usage: 88.0 MB


In [4]:
raw_df

,NORAD_CAT_ID,PERIOD,APOAPSIS,PERIAPSIS
EPOCH,,,,
2007-01-01 00:03:24.443999,22433,101.310997,893.202026,748.330994
2007-01-01 00:03:55.360511,17026,104.476997,1026.788940,913.116028
2007-01-01 00:04:25.817376,4993,103.421997,991.815979,849.054993
2007-01-01 00:05:07.286784,26207,100.232002,864.067993,675.075012
2007-01-01 00:05:40.624224,22408,102.888000,994.833984,795.719971
...,...,...,...,...
2009-12-31 23:59:22.953408,8158,114.183998,1472.323975,1364.348022
2009-12-31 23:59:27.523968,34505,100.186996,780.598999,754.239014
2009-12-31 23:59:33.565056,31486,102.253998,882.645996,848.080994


In [5]:
df = raw_df.reset_index().groupby(['NORAD_CAT_ID', pd.Grouper(key='EPOCH', freq=freq)]).mean()

In [6]:
all_dates = pd.date_range(start=df.index.get_level_values(1).min(), end=df.index.get_level_values(1).max(), freq=freq)

In [7]:
all_dates

DatetimeIndex(['2007-01-01', '2007-01-02', '2007-01-03', '2007-01-04',
               '2007-01-05', '2007-01-06', '2007-01-07', '2007-01-08',
               '2007-01-09', '2007-01-10',
               ...
               '2009-12-22', '2009-12-23', '2009-12-24', '2009-12-25',
               '2009-12-26', '2009-12-27', '2009-12-28', '2009-12-29',
               '2009-12-30', '2009-12-31'],
              dtype='datetime64[ns]', length=1096, freq='D')

In [8]:
df2 = df.unstack(level=0).reindex(all_dates).interpolate(method='linear', limit_area="inside", limit=30, limit_direction='both').stack().reset_index().rename(columns={'level_0':'EPOCH'})

In [9]:
df2

,EPOCH,NORAD_CAT_ID,PERIOD,APOAPSIS,PERIAPSIS
0,2007-01-01,51,118.208000,1681.223999,1519.645996
1,2007-01-01,53,118.320999,1685.048584,1526.050537
2,2007-01-01,85,104.206001,1342.460938,572.095459
3,2007-01-01,115,97.535004,669.490479,612.007996
4,2007-01-01,119,101.388000,903.093994,745.719971
...,...,...,...,...,...
7274244,2009-12-31,36092,101.236000,933.590454,700.828491
7274245,2009-12-31,36093,101.210999,933.750977,698.346985
7274246,2009-12-31,36099,101.939003,857.299988,843.664001
7274247,2009-12-31,36100,101.938004,857.721008,843.133972


In [10]:
df2.to_pickle(f"./gabbard_out_20072009_1d.pkl")